# Forecasting with Deep Learning models using GluonTS

Case study via the [M5 forecasting competition dataset](https://www.kaggle.com/competitions/m5-forecasting-accuracy).

M-competitions named after Spyros Makridakis, currently in their [6th edition](https://mofc.unic.ac.cy/the-m6-competition/).

M5 data provided by Walmart.

We assume the data set is downloaded locally (we can't provide it for Kaggle licensing).

## M5 dataset

* 42,840 hierarchical time series, 3049 products from 3 categories, 7 departments

* 3 US states: California (CA), Texas (TX), and Wisconsin (WI), 10 stores

* “Hierarchical” levels: item level, department level, product category level, and state level.

* Daily sales: Jan 2011 to June 2016. 

* included co-variates: prices, promotions, and holidays. 

* no missing values

### Loading the data

We use mainly standard pandas to load and manipulate data, for GluonTS models to use. 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint
from tqdm import tqdm
from gluonts.dataset.pandas import PandasDataset

In [ ]:
cal = pd.read_csv(
    Path("m5-forecasting-accuracy") / "calendar.csv",
)

weekly_prices = pd.read_csv(
    Path("m5-forecasting-accuracy") / "sell_prices.csv",
)

sales_and_features = pd.read_csv(
    Path("m5-forecasting-accuracy") / "sales_train_validation.csv",
)

In [ ]:
assert len(sales_and_features["item_id"].unique()) == 3049
assert len(sales_and_features["store_id"].unique()) == 10
assert len(sales_and_features) == 30490

In [ ]:
sales_and_features

Let's take a subset of this to make things a bit faster:

In [ ]:
sales_and_features = sales_and_features[sales_and_features.dept_id == "FOODS_3"]

We want to split the data into static (categorical features) vs dynamic (sales data). We keep the 'id' column in both, to be able to join the two. We also keep 'item_id' and 'store_id' in the sales data, to be able to join with prices later.

In [ ]:
features_columns = ["id", "dept_id", "cat_id", "store_id", "state_id"]
sales_columns = ["id", "item_id", "store_id"] + [f"d_{k}" for k in range(1, 1914)]

Split data into static (categorical features) vs dynamic (sales data).

In [ ]:
features = sales_and_features[features_columns].set_index("id").astype("category")
sales = sales_and_features[sales_columns]

In [ ]:
assert len(features) == 30490
assert len(features.columns) == 4
assert len(sales) == 30490
assert len(sales.columns) == 1916

Turn sales data into long format, to join with prices more easily.

In [ ]:
sales_long = sales.melt(id_vars=["id", "item_id", "store_id"], var_name="d", value_name="sales")

In [ ]:
weekly_prices

To join sales data with prices, first we add the `"wm_yr_wk"` column from `cal`. We also add the `"date"` column to build the time index. Then we join with `weekly_prices` on `"store_id"`, `"item_id"`, `"wm_yr_wk"`, to get the `"sell_price"` column in.

In [ ]:
temp = sales_long.merge(
    cal[["d", "wm_yr_wk", "date"]], on="d", how="left", suffixes=(None, "_right")
)

In [ ]:
sales_with_prices = temp.merge(weekly_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left", suffixes=(None, "_right"))

In [ ]:
sales_with_prices.index = pd.to_datetime(sales_with_prices["date"])

In [ ]:
len(sales_with_prices)

Some rows have missing price, which means the item was not for sale. Let's replace price there with some constant, and add a column indicating whether the product was for sale.

In [ ]:
sales_with_prices["for_sale"] = sales_with_prices["sell_price"].notna()
sales_with_prices["sell_price"].fillna(0.0, inplace=True)

Also we want to keep our target and feature columns as float32, to be compatible with the model later.

In [ ]:
sales_with_prices["sales"] = sales_with_prices["sales"].astype(np.float32)
sales_with_prices["sell_price"] = sales_with_prices["sell_price"].astype(np.float32)
sales_with_prices["for_sale"] = sales_with_prices["for_sale"].astype(np.float32)

In [ ]:
sales_with_prices

We're ready to construct our dataset object.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
dataset = PandasDataset.from_long_dataframe(
    sales_with_prices,
    item_id="id",
    target="sales",
    feat_dynamic_real=["sell_price", "for_sale"],
    static_features=features,
)

In [ ]:
len(dataset)

In [ ]:
dataset

In [ ]:
for entry in dataset:
    pprint(entry)
    break

## REMOVE: fake data just to try things out

In [ ]:
import pandas as pd
import numpy as np

dataset = [
    {
        "start": pd.Period("2022-03-04", freq="D"),
        "target": np.random.normal(size=1000),
        "feat_dynamic_real": np.random.normal(size=(2, 1000)).astype(np.float32),
        "feat_static_cat": np.array([0, 1, 2]),
    }
]

## A transformer model

We will train a transformer-based architecture (temporal fusion transformer, TFT?) on the above data.

Models in GluonTS are exposed as "estimator" objects: these take care of some data pre-processing (like replacing missing values in the data, and adding missing value indicator features, adding other calendar-related features, possibly other things depending on the model). An estimator is then trained with a training and validation datasets, and produces a "predictor" that contains the trained model to be used for prediction.

We will use a PyTorch-based estimator, which wraps the [Temporal Fusion Transformer model](https://arxiv.org/abs/1912.09363). This estimator relies on PyTorch Lightning for training, which means we can use all tooling available for it.

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
from gluonts.dataset.split import split
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator

In [ ]:
estimator = TemporalFusionTransformerEstimator(
    freq="1D",
    prediction_length=7,
    context_length=180,
    quantiles=[0.1, 0.5, 0.9],
    static_cardinalities=[1, 2, 3], #dataset.static_cardinalities.tolist(),
    dynamic_dims=[2],
    batch_size=32,
    trainer_kwargs={
        "max_epochs": 20,
        "logger": TensorBoardLogger("tb_logs"),
    }
)

Let's turn the dataset into a list, for faster iteration (good for training).

In [ ]:
validation_dataset = list(dataset)

In [ ]:
training_dataset, _ = split(validation_dataset, offset=-7)

In [ ]:
predictor = estimator.train(training_data=training_dataset, validation_data=validation_dataset)

In [ ]:
predictor

## Forecasting, evaluating, comparing

We will plot forecasts, evaluate accuracy and identify worst-cases, compare models.

## Hyperparameter Tuning

Tuning the model hyperparameters (a.g. architectural choices, number of layers, hidden layers sizes, etc.) is often important to get the best results. GluonTS does not provide model tuning features out of the box, but interfaces easily for dedicated toolboxes.

In [ ]:
import optuna

In [ ]:
from gluonts.ev.metrics import MASE
from gluonts.model.evaluation import evaluate_model

In [ ]:
training_dataset, validation_gen = split(dataset, offset=-7)
validation_data = validation_gen.generate_instances(prediction_length=7)

In [ ]:
df = evaluate_model(predictor, test_data=validation_data, metrics=[MASE()], seasonality=1)

In [ ]:
def tft_tuning_objective(trial):
    # get suggested hyperparameters values
    context_length = trial.suggest_int("context_length", 30, 180)
    variable_dim = trial.suggest_int("variable_dim", 10, 50)

    # set up model
    estimator = TemporalFusionTransformerEstimator(
        freq="1D",
        prediction_length=7,
        context_length=context_length,
        quantiles=[0.1, 0.5, 0.9],
        static_cardinalities=[1, 2, 3], #dataset.static_cardinalities.tolist(),
        dynamic_dims=[2],
        variable_dim=variable_dim,
        batch_size=32,
        trainer_kwargs={
            "max_epochs": 1,  # TODO set larger
        }
    )

    # train model
    predictor = estimator.train(training_dataset)

    # evaluate model
    df = evaluate_model(predictor, test_data=validation_data, metrics=[MASE()], seasonality=1)
    return df["MASE[0.5]"].iloc[0]
    

In [ ]:
study = optuna.create_study()

In [ ]:
res = study.optimize(tft_tuning_objective, n_trials=5)

## Datasets for experiments

Besides the specific M5 use-case above, it is important to validate the performance of a model class against multiple datasets. This is especially true when working on novel architectures, or adapting architectures from other domains (NLP, computer vision) to time series.